In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Sun Aug 28 17:25:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 9.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-t6lnm5il/farm-haystack_daee50e2d595472bba33473caffbf19c
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-t6lnm5il/farm-haystack_daee50e2d595472bba33473caffbf19c
  Resolved https://github.com/deepset-ai/haystack.git to commit 4e518cddddea79336eab1ce2f9ee61cf80df85da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 56.

In [ ]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.utils import fetch_archive_from_http

For probability distribution

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create dataframe with columns "title" and "text"
df = pd.read_csv(f"/content/drive/MyDrive/Dataset/Hello.csv", sep=",")
# Minimal cleaning
df.fillna(value="", inplace=True)
df.head() 

,title,text
0,Towards standardization of RNA quality assessment using user-independent cla...,Purity and integrity of RNA are critical elements for the overall success of...
1,Species-independent detection of RNA virus by representational difference an...,Identifying the causative agent of an infectious disease is the cornerstone ...
2,Factors affecting translation at the programmed −1 ribosomal frameshifting s...,The principal mechanism of translation is the accurate decoding of the tripl...
3,A universal BMV-based RNA recombination system—how to search for general rul...,A recombination is a very common phenomenon. It has been observed in all typ...
4,Engendering enthusiasm for sustainable disaster critical care response: why ...,"Disasters have been a part of human existence since ancient times, and so ha..."


In [ ]:
len(df)

102

In [ ]:
df.head()

,title,text
0,Towards standardization of RNA quality assessment using user-independent cla...,Purity and integrity of RNA are critical elements for the overall success of...
1,Species-independent detection of RNA virus by representational difference an...,Identifying the causative agent of an infectious disease is the cornerstone ...
2,Factors affecting translation at the programmed −1 ribosomal frameshifting s...,The principal mechanism of translation is the accurate decoding of the tripl...
3,A universal BMV-based RNA recombination system—how to search for general rul...,A recombination is a very common phenomenon. It has been observed in all typ...
4,Engendering enthusiasm for sustainable disaster critical care response: why ...,"Disasters have been a part of human existence since ancient times, and so ha..."


In [ ]:
new1.rename(columns={'metadata': 'title', 'body_text': 'text'}, inplace=True)
new1.head()

NameError: ignored

In [ ]:
# Use data to initialize Document objects
titles = list(df["title"].values)
texts = list(df["text"].values)
documents: List[Document] = []
for title, text in zip(titles, texts):
    documents.append(Document(content=text, meta={"name": title or ""}))

In [ ]:
import os

In [ ]:
# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
#document_store = FAISSDocumentStore(faiss_index_factory_str="Flat",embedding_dim=1596, return_embedding=True)
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)


# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


Downloading:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [ ]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)

Writing Documents:   0%|          | 0/102 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/100 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/112 [00:00<?, ? Docs/s]

In [ ]:
QUESTIONS =[
    
    "What is the reliable methods for identifying new agents?",
    "When China took measures of 'flu-like illness?",
    "What is maximum aqueous solution for DMSO?",
    "WHO many countries has been reported among IDUs",
    "What is the process of perceiving globalisation ?",
]

In [ ]:
"""QUESTIONS =[
            "Where does initial encounter with the host takes place?", #not so good output
            "Why was FRET used? ", #not so good output
            "What did geographical distance provide?", #great output
            "Can virus be transmitted on airplane?", #confused airplane with human airway
            "Do frequent infections early in life protect against the development of atopy?", #retrieved another passage related to infections
            "What are causes of calf disease?", #great output
            "When are cases of diarrhoea higher?", #showing infant diarrhoea cases
            "What is Hepatitis C virus?", #good output (could retrieve paper, not the passage)
            "How many people worldwide are persistently infected with hepatitis C virus?", #great output
            "Where is bacterial pathogens Neisseria meningitidis frequently found?", #great output
            "What is RA?", #confused RA with RV(rotavirus)
            "What is Rheumatoid arthritis?", #great output, exact passage retrieved
            "What is NCF-1?", #not so good output
            "What is Genome resequencing?" #great output, exact passage retrieved
]"""

In [ ]:
KEYWORDS=[
          "non-lymphatic origin", #not so good output
          "cytokine response", #great output, retrieved exact passage
          "Type I interferons", #Q1 till here    #perfect output, different passage 
          "FRET", #not so good output
          "dua replicon assay system", #good output, relative passage
          "cell-based screening", #Good output, relative passage
          "rotavirus vaccine", #great output, exact passage
          "rotavirus infection", #Q2 till here     #great output, exact passage
          "geographical distance", #great output, retrieved passage related to geography
          "microbial threats", #great output, exact passage
          "airplane", #confused airplane with human airways again
          "flights" #Q3 till here     #great output, different passage
]

In [ ]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 2}, "Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")


Query: What is the reliable methods for identifying new agents?
Answers:
[   {'answer': ' representational difference analysis'},
    {'answer': ' representational difference'}]

Query: When China took measures of 'flu-like illness?
Answers:
[{'answer': ' world health organization'}, {'answer': ' who'}]

Query: What is maximum aqueous solution for DMSO?
Answers:
[{'answer': ' drug discovery'}, {'answer': ' drug discovery and development'}]

Query: WHO many countries has been reported among IDUs
Answers:
[{'answer': ' zero'}, {'answer': ' 0'}]

Query: What is the process of perceiving globalisation ?
Answers:
[{'answer': ' social'}, {'answer': ' changes to your lifestyle'}]


In [ ]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline


p_retrieval = DocumentSearchPipeline(retriever)

for question in QUESTIONS:
  res = p_retrieval.run(query=question, params={"Retriever": {"top_k": 5}})
  print_documents(res, max_text_len=512)


Query: What is the reliable methods for identifying new agents?

{   'content': 'Since the emergence of SARS-CoV-2, a wide variety of '
               'diagnostic assays have been developed. These assays primarily '
               'use quantitative real-time reverse transcription polymerase '
               'chain reactions (qRT-PCRs) which detect viral RNA. Due to '
               'their high sensitivity and specificity, qRT-PCRs function as '
               'the gold standard for COVID-19 diagnostics [1]. However, '
               'qRT-PCR-based diagnostics require advanced laboratory '
               'infrastructure and trained personnel. Furthermore, the '
               'relatively long time to receive results could hamper...',
    'name': 'Rapid Antigen Test Performance with the SARS-CoV-2 Variants of '
            'Concern—Alpha, Beta, Gamma, Delta, and Omicron'}

{   'content': 'Identifying the causative agent of an infectious disease is '
               'the cornerstone for i

In [ ]:
#Checking with Keywords

p_retrieval = DocumentSearchPipeline(retriever)

for keywords in KEYWORDS:
  res = p_retrieval.run(query=keywords, params={"Retriever": {"top_k": 2}})
  print_documents(res, max_text_len=512)